In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)
os.environ['CUDA_VISIBLE_DEVICES'] = "4"
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

fish-gcp005
/home/akarshkumar0101/nca-alife/src
/home/akarshkumar0101/miniconda3/envs/nca-alife-jax/bin/python


In [2]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [3]:
import evosax
import jax
import jax.numpy as jnp
from jax.random import split

In [4]:
rng = jax.random.PRNGKey(0)
strategy = evosax.RandomSearch(popsize=2, num_dims=1)
es_params = strategy.default_params
state = strategy.initialize(rng, es_params)

for _ in range(1000):
    rng, _rng = split(rng)
    x, state = strategy.ask(_rng, state, es_params)
    fitness = x.mean(axis=-1)
    state = strategy.tell(x, fitness, state, es_params)

# Get best overall population member & its fitness
state.best_member, state.best_fitness

2024-08-09 08:10:46.013613: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


(Array([0.00258982], dtype=float32), Array(0.00258982, dtype=float32))

In [5]:
state

EvoState(mean=Array([0.00258982], dtype=float32), best_member=Array([0.00258982], dtype=float32), best_fitness=Array(0.00258982, dtype=float32), gen_counter=Array(1000, dtype=int32, weak_type=True))

In [6]:
import imageio
resize_fn = partial(jax.image.resize, shape=(512, 512, 3), method='nearest')

In [7]:
import flax.linen as nn
class DNCA(nn.Module):
    d_state: int = 16
    @nn.compact
    def __call__(self, x):
        x = jnp.pad(x, pad_width=1, mode='wrap')
        x = nn.Conv(features=48, kernel_size=(3, 3), padding="VALID")(x)
        x = nn.Conv(features=128, kernel_size=(1, 1))(x)
        x = nn.relu(x)
        x = nn.Conv(features=self.d_state, kernel_size=(1, 1))(x)
        return x

dnca = DNCA(d_state=16)

In [8]:
params = dnca.init(_rng, jnp.zeros((8, 8, 16)))

In [10]:
rng = jax.random.PRNGKey(0)

rng, _rng = split(rng)
color_map = jax.random.uniform(_rng, (16, 3))

rng, _rng = split(rng)
# state = jax.random.randint(_rng, (64, 64), minval=0, maxval=16)
state = jax.random.categorical(_rng, repeat(jax.nn.one_hot(0, 16)*6, "D -> H W D", H=64, W=64))

def step_dnca(state, _rng):
    state = jax.nn.one_hot(state, 16)
    logits = dnca.apply(params, state)
    state = jax.random.categorical(_rng, logits+state*10.)
    return state, state


state, statevid = jax.lax.scan(step_dnca, state, split(rng, 1000))

vid = (color_map[statevid]*255).astype(jnp.uint8)
vid = jax.vmap(resize_fn)(vid)

In [11]:
imageio.mimwrite(f'./temp/vid.gif', np.array(vid), fps=20)

(262144, 18)

In [40]:
2**18

262144

In [41]:
2**2

4